In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# Usable Code

## Scratch Work

In [2]:
!pwd

/Users/jseemayer/Documents/Me/Metis/NBA-Over-Under-Predictor


In [3]:
cd csv

/Users/jseemayer/Documents/Me/Metis/NBA-Over-Under-Predictor/csv


In [4]:
df = pd.read_csv('nba odds 2017-18.csv')

In [5]:
df.head()

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,...,2H,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,1017,501,V,Boston,19,19,33,28,99,214.5,...,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1017,502,H,Cleveland,29,25,18,30,102,4,...,110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1017,503,V,Houston,34,28,26,34,122,232,...,115.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1017,504,H,GoldenState,35,36,30,20,121,9,...,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1018,701,V,Brooklyn,30,33,35,33,131,212,...,112.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2624 entries, 0 to 2623
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         2624 non-null   int64  
 1   Rot          2624 non-null   int64  
 2   VH           2624 non-null   object 
 3   Team         2624 non-null   object 
 4   1st          2624 non-null   int64  
 5   2nd          2624 non-null   int64  
 6   3rd          2624 non-null   int64  
 7   4th          2624 non-null   int64  
 8   Final        2624 non-null   int64  
 9   Open         2624 non-null   object 
 10  Close        2624 non-null   object 
 11  ML           2624 non-null   int64  
 12  2H           2624 non-null   object 
 13  Unnamed: 13  0 non-null      float64
 14  Unnamed: 14  0 non-null      float64
 15  Unnamed: 15  0 non-null      float64
 16  Unnamed: 16  0 non-null      float64
 17  Unnamed: 17  0 non-null      float64
 18  Unnamed: 18  0 non-null      float64
 19  Unname

In [7]:
df = df.iloc[:,:13]

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2624 entries, 0 to 2623
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    2624 non-null   int64 
 1   Rot     2624 non-null   int64 
 2   VH      2624 non-null   object
 3   Team    2624 non-null   object
 4   1st     2624 non-null   int64 
 5   2nd     2624 non-null   int64 
 6   3rd     2624 non-null   int64 
 7   4th     2624 non-null   int64 
 8   Final   2624 non-null   int64 
 9   Open    2624 non-null   object
 10  Close   2624 non-null   object
 11  ML      2624 non-null   int64 
 12  2H      2624 non-null   object
dtypes: int64(8), object(5)
memory usage: 266.6+ KB


In [13]:
df1 = df.iloc[:4,:]

In [14]:
df1

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
0,1017,501,V,Boston,19,19,33,28,99,214.5,216,166,1.5
1,1017,502,H,Cleveland,29,25,18,30,102,4,4.5,-195,110
2,1017,503,V,Houston,34,28,26,34,122,232,231.5,364,115.5
3,1017,504,H,GoldenState,35,36,30,20,121,9,9,-470,2.5


## Scratch Work Graveyard